In [1]:
import pandas as pd
import pycountry
import dash
from dash import dcc
from dash import html
import plotly.express as px
from dash.dependencies import Input, Output


In [4]:
 #Load the CSV file
df = pd.read_csv('data/TemperatureData.csv')

# Function to convert alpha-2 country codes to alpha-3
def convert_alpha2_to_alpha3(alpha2_code):
    try:
        return pycountry.countries.get(alpha_2=alpha2_code).alpha_3
    except AttributeError:
        print(f"Kan de code niet vinden: {alpha2_code}")
        return None

# Apply the alpha-3 country codes
df['Country'] = df['Country'].apply(convert_alpha2_to_alpha3)

# Determine the minimum and maximum temperature in the dataset
min_temp = df['Temperature'].min()
max_temp = df['Temperature'].max()
df['Date'] = pd.to_datetime(df['Date'])

# Extract year and month from the date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [5]:
app = dash.Dash(__name__)

# Get a list of available years
available_years = df['Year'].unique()

app.layout = html.Div([
    dcc.Graph(id='choropleth'),
    dcc.Checklist(
        id='year-checkbox',
        options=[{'label': str(year), 'value': year} for year in available_years],
        value=[available_years[0]],  # Default to the first year
    ),
    dcc.Slider(
        id='month-slider',
        marks={month: str(month) for month in range(1, 13)},
        step=None,
    )
])

@app.callback(
    Output('month-slider', 'min'),
    Output('month-slider', 'max'),
    Output('month-slider', 'value'),
    Input('year-checkbox', 'value')
)
def update_slider(selected_years):
    min_month = df[df['Year'].isin(selected_years)]['Month'].min()
    max_month = df[df['Year'].isin(selected_years)]['Month'].max()
    initial_month = min_month
    return min_month, max_month, initial_month

@app.callback(
    Output('choropleth', 'figure'),
    Input('year-checkbox', 'value'),
    Input('month-slider', 'value')
)
def update_figure(selected_years, selected_month):
    filtered_df = df[(df['Year'].isin(selected_years)) & (df['Month'] == selected_month)]

    fig = px.choropleth(filtered_df, 
                        locations="Country", 
                        color="Temperature", 
                        hover_name="Country", 
                        color_continuous_scale='Viridis',  
                        range_color=(min_temp, max_temp))

    fig.update_geos(
        visible=True, resolution=50, scope="europe",
        showcountries=True, countrycolor="Black",
        showsubunits=True, subunitcolor="Blue"
    )

    fig.update_layout(
        title={
            'text': f'Temperatuurverdeling in Europa voor maand {selected_month} en jaar {selected_years}',
            'x': 0.5,  # Center the title
            'y': 0.98  # Adjust the vertical position of the title
        },
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
        height=1000, margin={"r": 0, "t": 0, "l": 0, "b": 0}
    )

    return fig
print("je vader")
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=7778)

je vader
